In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

Folders Path

In [ ]:
image_size = 150
labels = ['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
X_train = []
Y_train = []

for label in labels:
    for dataset_type in ['Training', 'Testing']:
        folder_path = f'../input/brain-tumor-classification-mri/{dataset_type}/{label}/'
        for img_name in os.listdir(folder_path):
            img = cv2.imread(os.path.join(folder_path, img_name))
            img = cv2.resize(img, (image_size, image_size))
            X_train.append(img)
            Y_train.append(label)

X_train = np.array(X_train)
Y_train = np.array(Y_train)

**# Shuffle and split the data**

In [ ]:
# Check the shapes before shuffling
print("Shapes before shuffling:", X_train.shape, Y_train.shape)

# Shuffle the data
X_train, Y_train = shuffle(X_train, Y_train, random_state=101)

# Check the shapes after shuffling
print("Shapes after shuffling:", X_train.shape, Y_train.shape)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.1, random_state=101)


**Convert labels to category**

In [ ]:
y_train = tf.keras.utils.to_categorical([labels.index(i) for i in y_train])
y_test = tf.keras.utils.to_categorical([labels.index(i) for i in y_test])
y_train = np.array(y_train)
y_test = np.array(y_test)

**Data Augmentation**

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [ ]:
# Create a TensorFlow Dataset
def load_and_preprocess_image(image_path, label):
    # Convert image_path to string
    image_path = tf.convert_to_tensor(image_path, dtype=tf.string)
    
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [image_size, image_size])
    img = img / 255.0  # Normalize to [0, 1]
    return img, label


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-7)

In [ ]:
# Dictionary to store models
models = {}

In [ ]:
# Custom Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_size, image_size, 3)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.3),
    Conv2D(64, (3, 3), activation='relu'),
    Conv2D(64, (3, 3), activation='relu'),
    Dropout(0.3),
    MaxPooling2D(2, 2),
    Dropout(0.3),
    Conv2D(128, (3, 3), activation='relu'),
    Conv2D(128, (3, 3), activation='relu'),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.3),
    Conv2D(128, (3, 3), activation='relu'),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.3),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(4, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
models['Custom_Model'] = model


In [ ]:
# AlexNet-like model
model = Sequential([
    Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(image_size, image_size, 3)),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    
    Conv2D(256, (5, 5), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    
    Conv2D(384, (3, 3), activation='relu', padding='same'),
    Conv2D(384, (3, 3), activation='relu', padding='same'),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    
    Dense(4096, activation='relu'),
    Dropout(0.5),
    
    Dense(4, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
models['AlexNet'] = model


In [ ]:
# VGG16
vgg16_weights_path = '/kaggle/input/vgg16-weights/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

vgg16 = tf.keras.applications.VGG16(weights=vgg16_weights_path, input_shape=(image_size, image_size, 3), include_top=False)
model = Sequential([
    vgg16,
    Flatten(),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(4, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
models['VGG16'] = model

In [ ]:
# ResNet50
resnet50_weights_path = '/kaggle/input/resnet50-weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

resnet50 = tf.keras.applications.ResNet50(weights=resnet50_weights_path, input_shape=(image_size, image_size, 3), include_top=False)

model = Sequential([
    resnet50,
    Flatten(),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(4, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
models['ResNet50'] = model


In [ ]:
# Xception
xception_weights_path = '/kaggle/input/xception-weights/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'
xception = tf.keras.applications.Xception(weights=xception_weights_path, input_shape=(image_size, image_size, 3), include_top=False)

model = Sequential([
    xception,
    Flatten(),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(4, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
models['Xception'] = model

In [ ]:
# InceptionV3
inceptionv3_weights_path = '/kaggle/input/inceptionv3-weights/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
inceptionv3 = tf.keras.applications.InceptionV3(weights=inceptionv3_weights_path, input_shape=(image_size, image_size, 3), include_top=False)
model = Sequential([
    inceptionv3,
    Flatten(),
    Dense(512, activation='relu'),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(4, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
models['InceptionV3'] = model

In [ ]:
# Train and evaluate models
evaluation_results = {}

# Set the number of epochs
num_epochs = 5

for model_name, model in models.items():
    print(f"Training and evaluating {model_name}...")
    
    # You can adjust the number of epochs easily here
    history = model.fit(
        datagen.flow(X_train, y_train, batch_size=32),
        steps_per_epoch=len(X_train) // 32,
        epochs=num_epochs,  
        validation_data=(X_test, y_test),
        callbacks=[early_stopping, reduce_lr]
    )

    # Rest of your evaluation code...


In [ ]:
# Obtain true labels
y_true = np.argmax(y_test, axis=1)

# Obtain predicted probabilities
y_pred_probs = model.predict(X_test)

# Obtain predicted labels
y_pred_classes = np.argmax(y_pred_probs, axis=1)

In [ ]:
 # Calculate evaluation metrics
accuracy = accuracy_score(y_true, y_pred_classes)
cm = confusion_matrix(y_true, y_pred_classes)
class_report = classification_report(y_true, y_pred_classes)

In [ ]:
 # Calculate TP, TN, FP, FN for each class
tp = np.diag(cm)
fp = np.sum(cm, axis=0) - tp
fn = np.sum(cm, axis=1) - tp
tn = np.sum(cm) - (tp + fp + fn)

In [ ]:
# Calculate additional metrics
precision = tp / (tp + fp)
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
f1_score = 2 * (precision * sensitivity) / (precision + sensitivity)
iou = tp / (tp + fp + fn)

In [ ]:
# Obtain predicted probabilities
y_pred_probs = model.predict(X_test)

# Initialize variables
fpr = {}
tpr = {}
roc_auc = {}

# Calculate ROC curve for each class
for i in range(len(labels)):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred_probs[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])


In [ ]:
# Store results
evaluation_results[model_name] = {
        'Accuracy': accuracy,
        'Confusion Matrix': cm,
        'True Positive': tp,
        'True Negative': tn,
        'False Positive': fp,
        'False Negative': fn,
        'Precision': precision,
        'Sensitivity': sensitivity,
        'Specificity': specificity,
        'F1 Score': f1_score,
        'IOU': iou,
        'ROC Curve': {'FPR': fpr, 'TPR': tpr, 'AUC': roc_auc},
    }

In [ ]:
 # Display evaluation results
print(f"\nEvaluation results for {model_name}:\n")
print(f"Accuracy: {accuracy:.2%}\n")
print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", class_report)
print("Precision:", precision)
print("Sensitivity (Recall):", sensitivity)
print("Specificity:", specificity)
print("F1 Score:", f1_score)
print("IOU:", iou)
print("ROC AUC:", roc_auc)

In [ ]:
# Display final results for all models
print("\nFinal Evaluation Results:\n")
for model_name, results in evaluation_results.items():
    print(f"{model_name}:\n")
    print(f"Accuracy: {results['Accuracy']:.2%}\n")
    print("Confusion Matrix:\n", results['Confusion Matrix'])
    print("\nROC AUC:\n", results['ROC Curve']['AUC'])
    print("\n\n")